In [ ]:
import pandas as pd
from tqdm import tqdm
from xml.dom import minidom
import re
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import STOPWORDS
import string
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# In this function documents are preprocessed 
def preprocessing(documents):
  new=[]
  for index, vals in tqdm(documents.iterrows()):
    document = vals['contents']
    document = document.lower()
    # Remove punctuation and numbers
    document = re.sub(r"[^A-Za-z\s]+", " ", document)
    # Remove extra XML query tag
    document = document.replace("query"," ")
    # Remove extra white spaces
    document = re.sub(r"[\s]{2,}", " ", document)
    # Tokenize string
    word_tokens = word_tokenize(document)
    # Remove stop words
    word_tokens = [token for token in word_tokens if token not in stop_words]
    word_tokens = [token for token in word_tokens if len(token) > 1]
    # Combining word tokens for results.
    results = " ".join(word_tokens)
    document = results.strip()
    new.append({'id': vals['id'], 'contents': document})
  new=pd.DataFrame(new)
  return new

In [ ]:
#In this function list of frequencies of words for a particular topic from the top k reranked documents are returned
def findwords(k, reranked,preprocessed):
  relIndex = {}
  for i in range(0, k):
    doc= reranked[i]
    text=preprocessed.loc[preprocessed["id"]==doc].iloc[0]["contents"]
    for term in text.split(" "):
      if term in relIndex.keys():
          relIndex[term] += 1
      else:
          relIndex[term] = 1
  return relIndex

In [ ]:
#In this function new queries are generated from the top_n frequent words of the top_k reranked documents 
#the parameter combined specifies if you want to combine all words in one query or not
def psuedo(topics,preprocessed,reranked,top_n,top_k,combined=True):
  final=[]
  for title, number in tqdm(zip(topics['Title'], topics['Number'])):
    d_list=[]
    d_list=reranked.loc[reranked["title"]==title].sort_values(by="score",ascending=False)["doc_id"].tolist()
    freq_words=findwords(top_k,d_list,preprocessed)
    sorted_words = sorted(freq_words.items(), key=lambda x:x[1], reverse=True)
    j=0
    new_title =title.lower()
    new_title =new_title.split(" ")
    new_title=[wordnet_lemmatizer.lemmatize(word) for word in new_title]
    new_title=' '.join(new_title)
    newQuery=title
    for i in range(100):
      term,frequency = sorted_words[i]
      if wordnet_lemmatizer.lemmatize(term) not in new_title:
        j=j+1
        newQuery +=  " "
        newQuery +=  term
        if combined==False:
          final.append({"Original":title,"New":newQuery,"Frequency":frequency})
          newQuery=title 
      if j==top_n:
        if combined==True:
          final.append({"Original":title,"New":newQuery,"Frequency":frequency})
        break
  return final

In [ ]:
topics=pd.read_csv("/content/drive/MyDrive/Touche/topics-task2.csv")
topics.head()

,Unnamed: 0,Number,Title,Objects
0,0,2,"Which is better, a laptop or a desktop?","laptop, desktop"
1,1,3,"Which is better, Canon or Nikon?","Canon, Nikon"
2,2,8,What are the advantages and disadvantages of P...,"PHP, Python"
3,3,9,Why is Linux better than Windows?,"Linux, Windows"
4,4,12,Train or plane? Which is the better choice?,"Train, plane"


In [ ]:
documents= pd.read_json(path_or_buf='/content/drive/MyDrive/Touche/touche-task2-passages-expanded-with-queries.jsonl', lines=True)
documents.head()

In [ ]:
reranked=pd.read_csv("/content/drive/MyDrive/Touche/Final_Reranked.csv")
reranked.head()

In [ ]:
preprocessed=preprocessing(documents)

In [ ]:
Final=psuedo(topics,preprocessed,reranked,1,20)
Final=pd.DataFrame(Final)
Final.head()

50it [01:11,  1.42s/it]


,Original,New,Frequency
0,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",39
1,"Which is better, Canon or Nikon?","Which is better, Canon or Nikon? vs",42
2,What are the advantages and disadvantages of P...,What are the advantages and disadvantages of P...,45
3,Why is Linux better than Windows?,Why is Linux better than Windows? software,12
4,Train or plane? Which is the better choice?,Train or plane? Which is the better choice? tr...,25


In [ ]:
topics=pd.read_csv("/content/drive/MyDrive/Touche/topics-task2.csv")
topics=topics.join(Final.set_index("Original"),on="Title")
topics=topics.drop("Unnamed: 0", axis=1)
topics.head()

,Number,Title,Objects,New,Frequency
0,2,"Which is better, a laptop or a desktop?","laptop, desktop","Which is better, a laptop or a desktop? computer",39
1,3,"Which is better, Canon or Nikon?","Canon, Nikon","Which is better, Canon or Nikon? vs",42
2,8,What are the advantages and disadvantages of P...,"PHP, Python",What are the advantages and disadvantages of P...,45
3,9,Why is Linux better than Windows?,"Linux, Windows",Why is Linux better than Windows? software,12
4,12,Train or plane? Which is the better choice?,"Train, plane",Train or plane? Which is the better choice? tr...,25


In [ ]:
topics.to_csv("/content/drive/MyDrive/Touche/topics-task2-psuedo_relevance.csv", index=False)